In [1]:
import torch

In [3]:
import numpy as np

In [34]:
data = np.load("./RealImpact/dataset/10_bowl/preprocessed/deconvolved_0db.npy")

In [35]:
data.shape

(3000, 208464)

In [36]:
data[0].shape

(208464,)

In [38]:
import numpy as np
from scipy.io.wavfile import write
import random


rate = 48000
ra = random.randint(0, 3000)
print(ra)
use_data = data[ra]
scaled = np.int16(use_data / np.max(np.abs(use_data)) * 32767)
# write('test.wav', rate, scaled)

2271


In [39]:
from IPython.display import Audio
wave_audio = scaled
Audio(wave_audio, rate=rate)

In [9]:
# !pip install ftfy regex tqdm
# !pip install git+https://github.com/openai/CLIP.git
!pip install transformers

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-s9r9ot70
  Running command git clone -q https://github.com/openai/CLIP.git /tmp/pip-req-build-s9r9ot70
Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 9.1 MB 10.4 MB/s eta 0:00:01
     |████████████████████████████████| 401 kB 30.9 MB/s eta 0:00:01
     |████████████████████████████████| 3.6 MB 27.3 MB/s eta 0:00:01
     |████████████████████████████████| 1.2 MB 69.2 MB/s eta 0:00:01
     |████████████████████████████████| 316 kB 41.2 MB/s eta 0:00:01
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [14]:
# import clip
import torch
from transformers import CLIPProcessor, CLIPModel, CLIP
from PIL import Image

In [5]:
clip.available_models()

['RN50',
 'RN101',
 'RN50x4',
 'RN50x16',
 'RN50x64',
 'ViT-B/32',
 'ViT-B/16',
 'ViT-L/14',
 'ViT-L/14@336px']

In [6]:
import os

# Load the model
device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load('ViT-B/32', device)

100%|███████████████████████████████████████| 338M/338M [01:15<00:00, 4.67MiB/s]


In [7]:
all_classes = ["wood", "ceramic", "glass", "plastic","iron", "steel", "polycarbonate"]

In [11]:
model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

config.json:   0%|          | 0.00/4.19k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/605M [00:00<?, ?B/s]

/opt/conda/lib/python3.9/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
2024-06-01 20:42:23.296781: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


preprocessor_config.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/592 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/862k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/525k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.22M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/389 [00:00<?, ?B/s]

In [20]:
texts

['a photo of wood',
 'a photo of ceramic',
 'a photo of glass',
 'a photo of plastic',
 'a photo of iron',
 'a photo of steel',
 'a photo of polycarbonate']

In [76]:
def infer(text_features, image_name):
    image = Image.open(image_name)
    
    inputs = processor(images=image, return_tensors="pt", padding=True)
    image_features = model.get_image_features(**inputs)
    
    probs = torch.softmax(image_features @ text_features.T, dim=1)
    return probs

In [74]:
texts = list(map(lambda x: f"a picture of an object made of {x}", all_classes))
text_input = processor(text = texts, return_tensors="pt", padding=True)
text_features = model.get_text_features(**text_input)

In [78]:
probs = infer(text_features, "test_image_bowl.png")
print("Top predictions:\n")
for idx, value in enumerate(probs[0]):
    print(f"{all_classes[idx]}: {100 * value.item():.2f}%")

Top predictions:

wood: 0.20%
ceramic: 53.86%
glass: 6.88%
plastic: 9.55%
iron: 0.41%
steel: 9.73%
polycarbonate: 19.38%


In [58]:
image_features.shape

torch.Size([1, 512])

## Flickr Materials Dataset - top k accuracy

In [1]:
from pathlib import Path
from tqdm import tqdm

In [2]:
all_classes = ["fabric", "foliage", "glass", "leather", "metal", "paper", "plastic", "stone", "water", "wood"]

In [91]:
texts = list(map(lambda x: f"a picture of an object made of {x}", all_classes))
text_input = processor(text = texts, return_tensors="pt", padding=True)
text_features = model.get_text_features(**text_input)

In [102]:
base_path = Path("flickr_materials", "image")

correct = [0]*10

for idx, cls in enumerate(all_classes):
    print(f"Working on {cls}")
    all_images = os.listdir(base_path.joinpath(cls))
    for img in tqdm(all_images, total=100):
        img_path = base_path.joinpath(cls, img)
        probs = infer(text_features, img_path)
        max_probs = probs.argmax(dim=1)
        if idx == max_probs:
            correct[idx] += 1

Working on fabric


100%|██████████| 100/100 [00:52<00:00,  1.91it/s]


Working on foliage


100%|██████████| 100/100 [00:48<00:00,  2.06it/s]


Working on glass


...

100%|██████████| 100/100 [00:47<00:00,  2.11it/s]


Working on water


100%|██████████| 100/100 [00:50<00:00,  1.97it/s]


Working on wood


100%|██████████| 100/100 [00:53<00:00,  1.86it/s]


In [103]:
for idx, value in enumerate(correct):
    print(f"Accuracy Score for {all_classes[idx]}: {value}%")

Accuracy Score for fabric: 92%
Accuracy Score for foliage: 87%
Accuracy Score for glass: 64%
Accuracy Score for leather: 75%
Accuracy Score for metal: 83%
Accuracy Score for paper: 80%
Accuracy Score for plastic: 76%
Accuracy Score for stone: 91%
Accuracy Score for water: 99%
Accuracy Score for wood: 65%


## Fine-Tuning on Flickr Dataset

In [1]:
!pip install transformers

Defaulting to user installation because normal site-packages is not writeable


In [1]:
import torch
from transformers import CLIPProcessor, CLIPModel
from PIL import Image
import numpy as np
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, random_split
import os
from pathlib import Path
import clip
from tqdm import tqdm

In [2]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device("cpu")
device

device(type='cuda')

In [3]:
model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

/opt/conda/lib/python3.9/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
2024-06-05 18:14:54.030629: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [36]:
def select_negative_example(pos):
    low = ((pos + 100) // 100) * 100 - pos
    high = 999 + (100) * (pos // 100)
    pos, low, pos+low, high, high % 1000
    n = 1
    return (torch.randint(2**63 - 1, size=(n, )) % (high - (pos+low)) + (pos+low)) % 1000

all_classes = os.listdir(Path("flickr_materials", "image"))
texts = list(map(lambda x: f"a picture of an object made of {x}", all_classes))
# text_input = processor(text=texts, return_tensors="pt", padding=True)


class CustomDataset(Dataset):
    def __init__(self):
        self.basepath = Path("flickr_materials", "image")
        self.classes = os.listdir(self.basepath)
        self.all_images = []
        for idx, nam in enumerate(self.classes):
            self.all_images.extend(os.listdir(self.basepath.joinpath(nam)))
            
        
    def __len__(self):
        return len(self.all_images)

    def __getitem__(self, idx):
        image_path = self.basepath.joinpath(self.classes[idx // 100], self.all_images[idx])
        curr_text = texts[idx // 100]
        curr_input = processor(images=Image.open(image_path), text=curr_text, return_tensors="pt", padding=True)
        curr_input = {k:v.squeeze().to(device) for k, v in curr_input.items()}
        
        return curr_input, idx

In [5]:
# text_features  -> (b, 512) x2 (pos, neg)
# image_features -> (b, 512) x2 (pos)
for params in model.text_model.children():
    params.requires_grad = False
    
for params in model.vision_model.children():
    params.requires_grad = False

In [6]:
def convert_models_to_fp32(model):
    for p in model.parameters():
        print(p)
        p.data = p.data.float()
        if p.grad is None: continue
        p.grad.data = p.grad.data.float()

In [12]:
model.to(device)

BATCH_SIZE = 32
dataset = CustomDataset()
train_dataset, val_dataset, _ = random_split(dataset, [0.9, 0.1], seed=42)
train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)
optimizer = torch.optim.Adam(model.parameters(), lr=5e-5,betas=(0.9,0.98),eps=1e-6,weight_decay=0.2)
loss_img = nn.CrossEntropyLoss()
loss_txt = nn.CrossEntropyLoss()

num_epochs = 30
best_val_loss = float('inf')

for epoch in range(num_epochs):
    total_loss = 0
    model.train()
    for batch in train_dataloader:
        optimizer.zero_grad()

        curr_input = batch
        output = model(**curr_input)
        logits_per_image, logits_per_text = output.logits_per_image, output.logits_per_text
        print(logits_per_image.shape[0], logits_per_text.shape, curr_input['input_ids'].shape)
        break

        # Compute loss
        ground_truth = torch.arange(BATCH_SIZE,dtype=torch.long,device=device)
        loss = (loss_img(logits_per_image,ground_truth) + loss_txt(logits_per_text,ground_truth))/2
        loss.backward()
        total_loss += loss.item()
        
        if device == torch.device("cpu"):
            optimizer.step()
        else:
            convert_models_to_fp32(model)
            optimizer.step()
            clip.model.convert_weights(model)
        print("First batch completed")
    print("first epoch completed")
    val_loss = 0
    model.eval()
    with torch.no_grad():
        for batch in val_dataloader:
            
            curr_input = batch
            output = model(**curr_input)
            logits_per_image, logits_per_text = output.logits_per_image, output.logits_per_text

            # Compute loss
            loss = (loss_img(logits_per_image,ground_truth) + loss_txt(logits_per_text,ground_truth))/2
            val_loss += loss.item()
            
    avg_val_loss = val_loss / len(val_dataloader)
    avg_train_loss = total_loss / len(train_dataloader)
    print(f"Epoch: [{epoch+1}/{num_epochs}], Loss: {avg_train_loss}, Val Loss: {avg_val_loss}")
    if best_val_loss > avg_val_loss:
        torch.save(model.state_dict(), Path("models", f"clip_ep_{epoch}_vl_{avg_val_loss:.2f}.pt"))    

32 torch.Size([32, 32]) torch.Size([32, 10])
first epoch completed


NameError: name 'ground_truth' is not defined

In [13]:
!pip install torchinfo

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Defaulting to user installation because normal site-packages is not writeable


In [15]:
from torchinfo import summary

In [16]:
text_input

NameError: name 'text_input' is not defined

## Inferencing using CLIP (with and without fine-tuning)

In [ ]:
base_model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

ft_model = CLIPModel.from_pretrained(Path("model"))
base_model.to(device)
ft_model.to(device)


In [38]:
BATCH_SIZE = 32
dataset = CustomDataset()
train_dataset, val_dataset = random_split(dataset, [0.9, 0.1])
val_dataloader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)

In [48]:
texts = list(map(lambda x: f"a picture of an object made of {x}", all_classes))
text_input = processor(text = texts, return_tensors="pt", padding=True)
text_input.to(device)
base_tf = base_model.get_text_features(**text_input)
ft_tf = ft_model.get_text_features(**text_input)
base_tf.to(device)
ft_tf.to(device)

base_true = 0
ft_true = 0

with torch.no_grad():
    for batch in val_dataloader:
        curr_input, idx = batch
        lab = idx // 100
        lab = lab.to(device)
        vi = curr_input['pixel_values']
        vf = base_model.get_image_features(pixel_values=vi)
        
        base_probs = torch.softmax(vf @ base_tf.T, dim=1)
        ft_probs = torch.softmax(vf @ ft_tf.T, dim=1)
        
        max_b_probs = base_probs.argmax(dim=1)
        max_ft_probs = ft_probs.argmax(dim=1)
        
        base_true += torch.sum(max_b_probs == lab).item()
        ft_true += torch.sum(max_ft_probs == lab).item()
        

In [49]:
base_true, ft_true

(84, 85)